## 自动求导

假设我们想对函数 $y=2 \mathbf{x}^{\top} \mathbf{x}$ 关于列向量 $x$ 求导

In [4]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算 $y$ 关于 $x$ 的梯度之前，我们需要一个地方来存储梯度

In [5]:
x.requires_grad_(True) # 等价于 `x = torch.arange(4.0, requires_grad=True) `
x.grad # 默认值是 None

现在让我们计算 $y$

In [6]:
y = 2 * torch.dot(x, x)
# 对于列向量 x, x^Tx 就是内积
# dot 函数就是求两个张量的点积
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算 `y` 关于 `x` 每个分量的梯度 

In [7]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad == 4 * x

tensor([True, True, True, True])

现在让我们计算 `x` 的另一个函数

In [9]:
# 在默认情况下，Pytorch 会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [10]:
# 对非标量调用 `backward` 需要传入一个 `gradient` 参数，该参数指定微分函数
x.grad.zero_()
y = x * x
# 等价于 y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [11]:
x.grad.zero_()
y = x * x
u = y.detach()
# 把 y detach 掉，就是把 y 变成一个相对于 x 的一个常数，而不是 y 关于 x 的函数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

即使构建函数的计算图需要通过 Python 控制流（例如条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [15]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
# 在这个函数中，虽然看控制了诸多条件，但是仍未 a 的线性变化函数，对于 a 的求导就是 f(a) / a 的值

a.grad == d / a

tensor(True)